In [1]:
import googleapiclient.discovery
import os
import pandas as pd
import time
from dotenv import load_dotenv

load_dotenv()

# Get api key from env
api_key = os.getenv('api_key')

youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey = api_key)

# Function to get channel id by username
def getChannelId(username):
    request = youtube.channels().list(
        part = 'snippet',
        forUsername = username
    )
    response = request.execute()

    channel_id = response['items'][0]['id']

    return channel_id

# Return json of channel data
def getUploadsId(channel_id):
    request = youtube.channels().list(
        part = 'contentDetails',
        id = channel_id
    )
    response = request.execute()

    uploads_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    return uploads_id

# Get video ids
def getVideoIds(uploads_id):
    request = youtube.playlistItems().list(
        part = 'snippet',
        playlistId = uploads_id,
        maxResults = 500
    )
    response = request.execute()

    # Get all video ids
    videos = {}
    for video in response['items']:
        videos[video['snippet']['resourceId']['videoId']] = video['snippet']['title']

    # Return as dict
    return videos

# Get comments
def getComments(video_id, num_comments, max_pages):

    # Comments
    comment_id = []
    text = []
    like_count = []
    publish_date = []

    next_page_token = None
    page_count = 0

    while page_count < max_pages:
        request = youtube.commentThreads().list(
            part = 'snippet',
            videoId = video_id,
            maxResults = num_comments,
            pageToken = next_page_token,
            textFormat = 'plainText'
        )
        
        response = request.execute()

        for item in response['items']:
            comment_id.append(item['id'])
            text.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
            like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
            publish_date.append(item['snippet']['topLevelComment']['snippet']['publishedAt'])

        next_page_token = response.get('nextPageToken')

        page_count += 1
        if not next_page_token:
            break
        time.sleep(1)

    return {'commentId': comment_id, 'text': text, 'likeCount': like_count, 'publishDate': publish_date}

In [2]:
username = 'MrBeast6000'
mr_beast_channelId = getChannelId(username)
uploads = getUploadsId(mr_beast_channelId)
videos = getVideoIds(uploads)

In [3]:
comment_dict = {
    'commentId': [],
    'likeCount': [],
    'publishDate': [],
    'text': []
}

for video_id, video_title in videos.items():
    print(f'Processing video: {video_title}')

    comments = getComments(video_id, 10, 3)
    comment_dict['commentId'].extend(comments['commentId'])
    comment_dict['likeCount'].extend(comments['likeCount'])
    comment_dict['publishDate'].extend(comments['publishDate'])
    comment_dict['text'].extend(comments['text'])

Processing video: Find This Key, Win $10,000
Processing video: World’s Fastest Robbers
Processing video: Would You Go On Vacation With A Stranger?
Processing video: Would You Risk Drowning for $500,000?
Processing video: Extreme Helmet Game
Processing video: $10,000 Human Shuffleboard
Processing video: Drift This Car, Win $10,000
Processing video: Watch This Video To Feed 1 Person In Need
Processing video: You Can Stop Child Labor
Processing video: How Fast Can I Go?
Processing video: Baseball Tic Tac Toe vs MLB Pro
Processing video: Could You Win This Obstacle Course?
Processing video: Find Your Dog, Win $10,000
Processing video: I Survived The 5 Deadliest Places On Earth
Processing video: This Is How The Pyramids Were Built
Processing video: I Crashed One Of The Most Expensive Cars In The World
Processing video: Last One To Fall Wins
Processing video: Last to Leave Their Circle Wins $500,000
Processing video: Highest Away From Me Wins $10,000
Processing video: Find The Human, Win $10

In [4]:
comment_df = pd.DataFrame(comment_dict)
comment_df.head()

,commentId,likeCount,publishDate,text
0,UgyTUhh6EAmdJkwQT_p4AaABAg,4685,2025-05-05T15:59:31Z,Order mom flowers on DoorDash with code MRBEAS...
1,Ugx51wSN_FipgAFQpA54AaABAg,0,2025-05-06T20:07:27Z,Mrbeast türk diyenler
2,UgzK8fTbjdx289VlNdR4AaABAg,0,2025-05-06T20:07:06Z,❤❤❤
3,UgzGcJSYo9kYQutLTH94AaABAg,0,2025-05-06T20:05:36Z,I am your fan
4,Ugz-W0jGgHjuiR-nIu94AaABAg,0,2025-05-06T20:04:11Z,Mister Beas


Export to csv

In [9]:
comment_df.to_csv('comments.csv', index = False)